In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sumarização de textos com Generative AI na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral
A sumarização de textos produz um resumo conciso de um texto. Existem dois tipos principais de resumo de texto: extrativo e abstrativo. A sumarização extrativa envolve selecionar frases críticas do texto original e combiná-las para formar um resumo. A sumarização abstrativa envolve a geração de novas sentenças que representam os pontos principais do texto original. Neste notebook, você passará por alguns exemplos de como LLM podem ajudar na geração de sumários de textos.

Saiba mais sobre resumo de texto na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/summarization-prompts).

### Objetivo

Neste tutorial, você aprenderá como usar LLM para sumarizar informações de texto trabalhando com os seguintes exemplos:
- Sumário da transcrição
- Sumarização de texto em bullets
- Sumário de diálogo com tarefas realizadas
- Tokenização de hashtag
- Geração de títulos e cabeçalhos

Você também aprenderá como avaliar resumos gerados por modelos comparando-os com resumos criados por humanos usando o `ROUGE` como uma estrutura de avaliação.

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Sumarização de texto

### Sumarizar uma transcrição

Neste primeiro exemplo, você resume um texto sobre computação quântica.

In [ ]:
prompt = """
Forneça um resumo muito curto, não mais do que três frases, para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Resumo:

"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

Em vez de um resumo, podemos pedir um TL;DR (*"too long; didn't read"* ou "muito longo; não li" em tradução livre). Você pode comparar as diferenças entre as saídas geradas.

In [ ]:
prompt = """
Forneça um TL;DR para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

TL;DR:
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Sumarizar em bullets
No exemplo a seguir, você usa o mesmo texto sobre computação quântica, mas pede ao modelo para sumarizar com bullets. Sinta-se à vontade para alterar o prompt.

In [ ]:
prompt = """
Forneça um resumo muito curto em quatro bullets para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Bullets:

"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text
)

### Sumarizar diálogos com tarefas realizadas
O sumário do diálogo envolve condensar uma conversa em um formato mais curto para que você não precise ler toda a discussão, mas possa aproveitar um resumo. Neste exemplo, você pede ao modelo para resumir uma conversa de exemplo entre um cliente de varejo online e um agente de suporte e incluir tarefas no final.

In [ ]:
prompt = """
Gere um resumo da conversa a seguir e, no final, resuma as tarefas para o agente de suporte:

Cliente: Olá, sou José e recebi o item errado.

Agente de suporte: Olá, José. Como você gostaria de ver isso resolvido?

Cliente: Eu quero devolver o item e obter um reembolso, por favor.

Agente de Suporte: Claro. Posso processar o reembolso para você agora. Posso ter o número do seu pedido, por favor?

Cliente: É [NÚMERO DO PEDIDO].

Agente de suporte: Obrigado. Processei o reembolso e você receberá seu dinheiro de volta em 14 dias.

Cliente: Muito obrigado.

Agente de suporte: De nada, José. Tenha um bom dia!

Resumo:
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text
)

### Tokenização de hashtag
A tokenização de hashtag é o processo de pegar um pedaço de texto e obter os "tokens" de hashtag. Você pode usar isso, por exemplo, se quiser gerar hashtags para suas campanhas de mídia social. Neste exemplo, você pega [este tweet do Google Cloud](https://twitter.com/googlecloud/status/1649127992348606469) e gera algumas hashtags que pode usar.

In [ ]:
prompt = """
Tokenize as hashtags deste tweet:

Google Cloud
@googlecloud
Como os dados podem ajudar nosso planeta em mudança? 🌎

Em homenagem ao #EarthDay neste fim de semana, estamos orgulhosos de compartilhar como estamos fazendo parceria com
@ClimateEngine para aproveitar o poder dos dados geoespaciais e direcionar para um futuro mais sustentável.

Confira como → https://goo.gle/3mOUfts
"""

print(
    generation_model.predict(prompt, temperature=0.8, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Geração de títulos e cabeçalhos
Abaixo, você pede ao modelo para gerar cinco opções para possíveis combinações de título/cabeçalho para um determinado trecho de texto.

In [ ]:
prompt = """
Escreva um título para este texto, me dê cinco opções:
Seja ajudando médicos a identificar doenças ou encontrando fotos de “abraços”, a IA está por trás de muito do trabalho que fazemos no Google. E em nosso Arts & Culture Lab em Paris, temos experimentado como a IA pode ser usada em benefício da cultura.
Hoje, estamos compartilhando nossos últimos experimentos - protótipos que se baseiam em sete anos de trabalho em parceria com 1.500 instituições culturais em todo o mundo.
Cada um desses aplicativos experimentais executa algoritmos de IA em segundo plano para permitir que você descubra conexões culturais escondidas em arquivos e até mesmo encontre obras de arte que combinem com a decoração da sua casa."
"""

print(
    generation_model.predict(prompt, temperature=0.8, max_output_tokens=256, top_k=1, top_p=0.8).text
)

## Avaliação
Você pode avaliar os resultados das tarefas de resumo usando [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) como uma estrutura de avaliação. `ROUGE` (Recall-Oriented Understudy for Gisting Evaluation) são medidas para determinar automaticamente a qualidade de um resumo comparando-o com outros resumos (ideais) criados por humanos. As medidas contam o número de unidades sobrepostas, como n-gram, sequências de palavras e pares de palavras entre o resumo gerado por computador a ser avaliado e os resumos ideais criados por humanos.


O primeiro passo é instalar a biblioteca `ROUGE`.

In [ ]:
!pip install rouge

Crie um resumo a partir de um LLM que você pode usar para comparar com um resumo gerado por humanos.

In [ ]:
from rouge import Rouge

ROUGE = Rouge()

prompt = """
Forneça um resumo muito curto, máximo de quatro frases, para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Resumo:

"""

candidate = generation_model.predict(prompt, temperature=0.1, max_output_tokens=1024, top_k=40, top_p=0.9).text

print(candidate)

You will also need a human-generated summary that we will use to compare to the `candidate` generated by the model. We will call this `reference`. 

In [ ]:
reference = "Computadores quânticos são sensíveis a ruídos e erros. Para preencher essa lacuna, precisaremos de correção de erros quânticos. A correção de erros quânticos protege as informações codificando vários qubits físicos para formar um 'qubit lógico'."

Agora você pode pegar o candidato e a referência para avaliar o desempenho. Neste caso, ROUGE lhe dará:

- `rouge-1`, que mede a sobreposição de unigramas
- `rouge-2`, que mede a sobreposição de bigramas
- `rouge-l`, que mede a maior subsequência comum

In [ ]:
ROUGE.get_scores(candidate, reference)